# Project Three - Frisbee Flight
Adi Sudhakar and George Tighe

In [11]:
from modsim import *

In [12]:
launch = SweepSeries()
launch = linrange(0, 95, 5)
aattack = SweepSeries()
aattack = linrange(-10, 15, 5)
print(launch)
print(aattack)

[ 0  5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90]
[-10  -5   0   5  10]


In [13]:
system = System(m = 0.175, d = 0.27, v0 = 15, launch = launch, aattack = aattack, g = -9.8, 
                p_air = 1.225)
state = State(xpos = 0, zpos = 1.2, vx = 11, vz = 10)

In [9]:
def drag(system, state):
    unpack(system)
    a0 = -4 #Deg. Angle of attack at minimum drag constant. Determined experimentally.
    cd0 = 0.1769 #Determined experimentally. Skin friction and pressure drag.
    cda = 0.685 #Determined experimentally. Induced drag.
    A = 0.057 #m^2. Planform area of the frisbee
    cdrag = cd0 + cda*(aattack-a0)**2 #Drag coefficient.
    fdrag = (1/2)*p_air*A*cdrag*(v0**2) #Drag force.
    print(cdrag)
    return fdrag

In [10]:
print(drag(system, state))

[ 24.8369   0.8619  11.1369  55.6619 134.4369]
[ 195.10161103    6.77049384   87.48382978  437.24161884 1056.04386103]


In [6]:
def lift(system, state):
    unpack(system)
    cl0 = 0.3331 #Lift for 0 angle of attack.
    cla = 1.9124 #Additional lift due to increased angle of attack.
    A = 0.057 #m^2. Planform area of the frisbee
    clift = cl0 + (cla*aattack) #Coefficient of lift.
    flift = (1/2)*p_air*A*clift*v0**2 #Lift force.
    return flift

In [7]:
def updatethefunc(system, state, t):
    unpack(system)
    fvert = lift(system, state) - m*g
    dvz = fvert * (t-(t-1)) / m
    dvx = drag(system, state) / m
    vzf = dvz + state.vz
    vxf = dvx + state.vx
    xposf = state.xpos + (state.vx*(t+(t-1)))
    zposf = state.zpos + (state.vz*(t+(t-1)))
    newstate = State(xpos = xposf, zpos = zposf, vx = vxf, vz = vzf)
    return newstate

In [7]:
print(updatethefunc(system, state, 5))

xpos                                                   99
zpos                                                 91.2
vx      [1125.8663487500005, 49.68853625000001, 510.90...
vz      [-823.6765237500002, -394.46224875000013, 34.7...
dtype: object


In [ ]:
def runsimulation():
    frame = TimeFrame(columns = state.index)
    frame.row[0] = state
    ts = linrange(0, 100, 1)
    for t in ts:
        frame.row[t+1] = updatethefunc(system, frame.row, t)
    return frame